Since switching to the darkcount model, I wanted to check the difference between the experimental data adn the calculated darkcounts values. We should compare these to see if we are ok with these errors. (How do we discern which option-- experimental subtraction or the model data subtraction is "better"?)

In [1]:
import os
import numpy as np
import h5py
import matplotlib.pyplot as plt
from datetime import datetime
from reportlab.lib.pagesizes import letter, landscape
from reportlab.pdfgen import canvas
from reportlab.lib.utils import ImageReader
from reportlab.lib import colors
from reportlab.platypus import Table, TableStyle
import io

def compare_darkcounts(directory, experiment_title, bias_file='darkcounts_combined_b2.npy', slope_file='darkcounts_combined_Sd2.npy', base_data_folder="data"):
    # Load bias and slope arrays
    bias = np.load(os.path.join('./data/', bias_file))
    slope = np.load(os.path.join('./data/', slope_file))

    # Get the list of darkcount files
    darkcount_files = [f for f in os.listdir(directory) if f.startswith('darkcounts')]
    darkcount_files.sort(key=lambda x: float(x[10:-3]))

    # Read the darkcount files and store the data
    measured_darkcounts = []
    exposure_times = []
    for file in darkcount_files:
        file_path = os.path.join(directory, file)
        with h5py.File(file_path, 'r') as h5f:
            darkcount = h5f['Cube']['Images'][()]
            exposure_time = h5f['Cube']['TimeExposure'][()].item()
            measured_darkcounts.append(darkcount)
            exposure_times.append(exposure_time)

    measured_darkcounts = np.squeeze(np.array(measured_darkcounts))
    exposure_times = np.array(exposure_times)

    # Calculate modeled darkcounts
    modeled_darkcounts = bias[np.newaxis, :, :] + slope[np.newaxis, :, :] * exposure_times[:, np.newaxis, np.newaxis]

    # Calculate difference
    difference = measured_darkcounts - modeled_darkcounts

    # Calculate statistics
    mean_diff = np.mean(difference, axis=(1, 2))
    std_diff = np.std(difference, axis=(1, 2))
    max_diff = np.max(np.abs(difference), axis=(1, 2))

    # Generate PDF report
    experiment_folder = os.path.basename(os.path.normpath(directory))
    data_folder = os.path.join(base_data_folder, experiment_folder)
    os.makedirs(data_folder, exist_ok=True)

    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"darkcount_comparison_report_{experiment_title}_{timestamp}.pdf"
    filepath = os.path.join(data_folder, filename)

    c = canvas.Canvas(filepath, pagesize=landscape(letter))
    width, height = landscape(letter)

    # Add title
    c.setFont("Helvetica-Bold", 16)
    c.drawString(50, height - 50, f"Darkcount Comparison Report: {experiment_title}")

    # Add timestamp
    c.setFont("Helvetica", 12)
    c.drawString(50, height - 70, f"Generated on: {timestamp}")

    y_position = height - 100

    # Create table data
    table_data = [['Exposure Time (s)', 'Mean Difference', 'Std Dev of Difference', 'Max Absolute Difference']]
    for time, mean, std, max_abs in zip(exposure_times, mean_diff, std_diff, max_diff):
        table_data.append([f"{time:.4g}", f"{mean:.4g}", f"{std:.4g}", f"{max_abs:.4g}"])

    # Create table
    table = Table(table_data)
    table.setStyle(TableStyle([
        ('BACKGROUND', (0, 0), (-1, 0), colors.grey),
        ('TEXTCOLOR', (0, 0), (-1, 0), colors.whitesmoke),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 0), (-1, 0), 'Helvetica-Bold'),
        ('FONTSIZE', (0, 0), (-1, 0), 12),
        ('BOTTOMPADDING', (0, 0), (-1, 0), 12),
        ('BACKGROUND', (0, 1), (-1, -1), colors.beige),
        ('TEXTCOLOR', (0, 1), (-1, -1), colors.black),
        ('ALIGN', (0, 0), (-1, -1), 'CENTER'),
        ('FONTNAME', (0, 1), (-1, -1), 'Helvetica'),
        ('FONTSIZE', (0, 1), (-1, -1), 10),
        ('TOPPADDING', (0, 1), (-1, -1), 6),
        ('BOTTOMPADDING', (0, 1), (-1, -1), 6),
        ('GRID', (0, 0), (-1, -1), 1, colors.black)
    ]))

    # Draw the table
    table.wrapOn(c, width - 100, height)
    table.drawOn(c, 50, y_position - table._height)

    y_position -= (table._height + 50)

    # Plot results
    fig, axs = plt.subplots(2, 2, figsize=(15, 15))
    fig.suptitle(f'Darkcount Comparison for {experiment_title}')

    axs[0, 0].plot(exposure_times, mean_diff, 'o-')
    axs[0, 0].set_xlabel('Exposure Time (s)')
    axs[0, 0].set_ylabel('Mean Difference')
    axs[0, 0].set_title('Mean Difference vs Exposure Time')

    axs[0, 1].plot(exposure_times, std_diff, 'o-')
    axs[0, 1].set_xlabel('Exposure Time (s)')
    axs[0, 1].set_ylabel('Standard Deviation of Difference')
    axs[0, 1].set_title('Std Dev of Difference vs Exposure Time')

    axs[1, 0].plot(exposure_times, max_diff, 'o-')
    axs[1, 0].set_xlabel('Exposure Time (s)')
    axs[1, 0].set_ylabel('Max Absolute Difference')
    axs[1, 0].set_title('Max Absolute Difference vs Exposure Time')

    axs[1, 1].hist(difference[-1].flatten(), bins=50)
    axs[1, 1].set_xlabel('Difference')
    axs[1, 1].set_ylabel('Frequency')
    axs[1, 1].set_title(f'Histogram of Differences (Exposure Time: {exposure_times[-1]:.2f}s)')

    plt.tight_layout()
    
    # Save plot to a bytes buffer
    buf = io.BytesIO()
    plt.savefig(buf, format='png', dpi=300, bbox_inches='tight')
    buf.seek(0)
    
    # Add plot to PDF
    img = ImageReader(buf)
    img_width = width - 100
    img_height = img_width * (img.getSize()[1] / img.getSize()[0])  # Maintain aspect ratio
    c.drawImage(img, 50, y_position - img_height, width=img_width, height=img_height)
    
    plt.close(fig)

    # Add darkcount images
    c.showPage()
    y_position = height - 50

    for i, (measured, modeled, diff) in enumerate(zip(measured_darkcounts, modeled_darkcounts, difference)):
        if y_position - 300 < 50:  # Check if there's enough space on the page
            c.showPage()
            y_position = height - 50

        c.setFont("Helvetica-Bold", 12)
        c.drawString(50, y_position, f"Exposure Time: {exposure_times[i]:.4g}s")
        y_position -= 20

        fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15, 5))
        
        # Measured Darkcount
        im1 = ax1.imshow(measured, cmap='gray')
        ax1.set_title('Measured Darkcount')
        ax1.axis('off')
        plt.colorbar(im1, ax=ax1, label='Intensity')
        
        # Modeled Darkcount
        im2 = ax2.imshow(modeled, cmap='gray')
        ax2.set_title('Modeled Darkcount')
        ax2.axis('off')
        plt.colorbar(im2, ax=ax2, label='Intensity')
        
        # Difference
        max_diff = np.max(np.abs(diff))
        im3 = ax3.imshow(diff, cmap='bwr', vmin=-max_diff, vmax=max_diff)
        ax3.set_title('Difference')
        ax3.axis('off')
        plt.colorbar(im3, ax=ax3, label='Difference')

        plt.tight_layout()

        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=300, bbox_inches='tight')
        buf.seek(0)
        plt.close(fig)

        img = ImageReader(buf)
        img_width = width - 100
        img_height = img_width * (img.getSize()[1] / img.getSize()[0])  # Maintain aspect ratio
        c.drawImage(img, 50, y_position - img_height, width=img_width, height=img_height)
        y_position -= (img_height + 50)

    c.save()
    print(f"Darkcount comparison report saved: {filepath}")

    return measured_darkcounts, modeled_darkcounts, difference, exposure_times

# Usage
directory = '/Users/allisondennis/Library/CloudStorage/OneDrive-NortheasternUniversity/Shared Documents - Dennis Lab/Image processing/IR VIVO data/AMD/240329_Water_immersed'
experiment_title = '240329_Water_immersed'
measured, modeled, diff, times = compare_darkcounts(directory, experiment_title)

Darkcount comparison report saved: data/240329_Water_immersed/darkcount_comparison_report_240329_Water_immersed_20240806_080245.pdf
